# Proyect_02b

### Introduction 
___
_Objective:_ __To explore, segment and group the neighbourhoods of the city of Toronto.__   
The information is obtained with web scraping by Wikipedia and the data is transformed into a structured data format.   
The project is divided into 3 parts to facilitate the understanding and implementation of the code.

### Table of contents 
#### Notebook 2
This notebook focuses on scraping the information, cleaning it up and setting up a data frame for later exploration.
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Import Data</a></li>
        <li><a href="#ref2">Geolocation</a></li>
        <li><a href="#ref3">Resources</a></li>
    </ol>
</div>
<br>

<a id="ref1"></a>
# 1. Import 
In this section installs and imports the necessary packages for the project.
***

In [1]:
!pip install beautifulsoup4 # HTML and XML data extraction library.
!pip install request # requests , timeout
# Importing Packages
import pandas as pd # DataFrame
import numpy as np # Arrays
from bs4 import BeautifulSoup # HTML and XML data extraction library.
import requests # requests , timeout
print("Ready")

Ready


<a href="https://github.com/Azhura/Coursera_Capstone/blob/master/Project02a.ipynb">Notebook 1: Data Wrangling</a><br>

In [2]:
# Code - Notebook 1
# Loading the url data.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(url,timeout=5) # requests , timeout
soup = BeautifulSoup(page_response.content,'lxml') # Transforming to BeautifulSoup object
table = soup.find_all('table')[0] # Filtering the html data table
df = pd.read_html(str(table))[0] # Transforming data with pandas
# Value Filtering
na = df['Borough'] != 'Not assigned' 
df_na = df[na] 
new_data = df_na
new_data['Neighborhood'] = new_data['Neighborhood'].str.replace('/',',')
new_data  =  new_data.sort_values (['Postal code','Borough','Neighborhood'])
new_data.index = np.arange(0, len(new_data)) # changing start index
print("Size: ",new_data.shape)
new_data.head() # Head After

Size:  (103, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<a id="ref2"></a>
# 2. Geolocation
I tried to get the information with the geodecoder library but it doesn't seem to be working.    

In this section the csv of the following link http://cocl.us/Geospatial_data,       
is loaded to obtain the latitudes and longitudes of toronto,      
adding the information to the first data frame.    
***

In [3]:
# IBM Cloud Code -> Read - Geospatial_Coordinates.csv
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_01ba54beae4e4c2ba72338f0bffbcfc0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WHQpda1UemxiKh1dWr6ktIildCxStzrXvpilUjPCWLCL',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_01ba54beae4e4c2ba72338f0bffbcfc0.get_object(Bucket='datascience-donotdelete-pr-rxrupxu6gwy6yg',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
#------------------------------End -> IBM Cloud Code-------------------------------
# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
print("Data Ready")
print("size: ",df_data_0.shape)
df_data_0.head()

Data Ready
size:  (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
# Joining data frames by index.
new_data02 = new_data.join(df_data_0, lsuffix='index', rsuffix='index')
new_data02.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [5]:
# Removing the repeated column
new_data02.drop(['Postal Code'],axis=1,inplace=True)
new_data02.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [6]:
new_data02.shape

(103, 5)

<div class="alert alert-block alert-warning" style="margin-top: 20px"><b>To be continued in Notebook 3</b></div><br>
<a href="https://github.com/Azhura/Coursera_Capstone/blob/master/Project02a.ipynb">Notebook 1: Data Wrangling</a><br>
<a href="https://github.com/Azhura/Coursera_Capstone/blob/master/Project02b.ipynb">Notebook 2: Geolocation</a><br>
<a href="https://github.com/Azhura/Coursera_Capstone/blob/master/Project02c.ipynb">Notebook 3: GeoData Exploration</a>

<a id="ref3"></a>
# 3. Resources
<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="https://www.coursera.org/learn/applied-data-science-capstone">Applied Data Science Capstone</a></li>
    <li><a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">Beautiful Soup Documentation</a></li>
    <li><a href="https://requests.readthedocs.io/en/master/">Request Documentation</a></li>
    <li><a href="https://pandas.pydata.org/docs/">Pandas Documentation</a></li>
    <li><a href="https://numpy.org/doc/">NumPy Documentation</a></li>
    <li><a href="https://github.com/Azhura/Coursera_Capstone">Labs</a></li>
</ol>
</div>

This notebook was created by [Carlos Alberto Gómez Prado](https://www.linkedin.com/in/carlospradobigdata/), as an assignment for the IBM coursera course.   

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://www.coursera.org/professional-certificates/ibm-data-science).   
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).

---